In [15]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import unidecode
import json
from os import listdir

In [20]:

MYSQL_CONFIG = {
    'USER': 'ari',
    'PASSWORD': '',
    'PORT': '3306',
    'DATABASE': 'growing_instability',
    'HOST': '35.184.108.36'
}

engine = create_engine('mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}'.format(
    user=MYSQL_CONFIG['USER'], 
    password=MYSQL_CONFIG['PASSWORD'],
    port=MYSQL_CONFIG['PORT'],
    dbname=MYSQL_CONFIG['DATABASE'],
    host=MYSQL_CONFIG['HOST']
))
pd.read_sql_query("""
ALTER TABLE `growing_instability`.`training_data` 
CHANGE COLUMN `id` `id` TEXT NULL DEFAULT NULL FIRST;
""", engine)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



InternalError: (pymysql.err.InternalError) (1170, "BLOB/TEXT column 'id' used in key specification without a key length") [SQL: '\nALTER TABLE `growing_instability`.`training_data` \nCHANGE COLUMN `id` `id` TEXT NULL DEFAULT NULL FIRST;\n']

In [3]:
ORIGIN_FOLDER = '../origin/'

In [4]:
# Load topics
topics = pd.read_csv(ORIGIN_FOLDER + 'topicDictionary.csv',header=None)
topics.columns = ['topic']
topics.to_sql('topics', engine, if_exists='replace',index=False)

In [5]:
sampleSubmission = pd.read_csv(ORIGIN_FOLDER + 'sampleSubmission.csv')
sampleSubmission.to_sql('sample_submission', engine, if_exists='replace',index=False)

In [6]:
def loadDataJsonFormatIntoTable(location, mainKey='TestData'):
    with open(location) as data_file:    
        data = json.load(data_file)
        return pd.DataFrame([
            {
                'id': dId,
                'bodyText': unidecode.unidecode(data[mainKey][dId]['bodyText']),
                'webPublicationDate': data[mainKey][dId]['webPublicationDate']   
            }
            for dId in data[mainKey] 
        ]);


In [14]:
loadDataJsonFormatIntoTable(ORIGIN_FOLDER + 'testData.json').to_sql('test_data',engine, if_exists='replace',index=False,chunksize=10)

In [8]:
tData = loadDataJsonFormatIntoTable(ORIGIN_FOLDER + 'trainingData/1999a_TrainingData.json', "TrainingData")

#.to_sql('test_data',engine, if_exists='replace',index='id')

In [22]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    pool = Pool(processes=1)              # Start a worker processes.
    result = pool.apply_async(f, [10], callback) 